## Setup

In [96]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect

In [97]:
# Create Database Connection
engine = create_engine("sqlite:///../Resources/mammal_masses.sqlite", echo=False)

In [98]:
# Reflect Database into ORM classes
Base = automap_base()
Base.prepare(autoload_with=engine)
Base.classes.keys()

['sa', 'aus', 'af', 'ea', 'oceanic', 'insular', 'na']

In [99]:
# Map Europe class
EA = Base.classes.ea

In [100]:
# Map North American class
NA = Base.classes.na

In [101]:
# create a session
session = Session(engine)

## Filtering Review

Filters are the "WHERE" clause for your select statement. 

In [102]:
# Filter North American mammals whose genus is "Antilocapra"
# Query, loop over and print out animals.
mammals = session.query(NA).filter(NA.genus == 'Antilocapra').all()
for mammal in mammals:
    print(f"Family: {mammal.family}, Genus: {mammal.genus}")

Family: Antilocapridae, Genus: Antilocapra


## Joins

A SQL join combines columns from one or more tables in a relational database. 

It creates a set that can be saved as a table or used as it is. 

A JOIN is a means for combining columns from one (self-table) or more tables by using values common to each.

In [103]:
# Get the table names using `inspect()`.
inspector = inspect(engine)
inspector.get_table_names()

['af', 'aus', 'ea', 'insular', 'na', 'oceanic', 'sa']

In [104]:
# Get a list of column names and types
columns = inspector.get_columns('ea')
for c in columns:
    print(c['name'], c["type"])

id INTEGER
continent TEXT
status TEXT
order TEXT
family TEXT
genus TEXT
species TEXT
log_mass FLOAT
combined_mass FLOAT
reference TEXT


In [115]:
# Create an inner join
inner_join = session.query(NA, EA).join(NA, EA.order == NA.order).limit(200).all()

In [119]:
# Display specific columns of the result
for na_record, ea_record in inner_join:
    # Access columns of the NA table
    na_order = na_record.order
    na_family = na_record.family
    na_genus = na_record.genus
    na_species = na_record.species

    # Access columns of the EA table
    ea_order = ea_record.order
    ea_family = ea_record.family
    ea_genus = ea_record.genus
    ea_species = ea_record.species

    # Print or use the values as needed
    print(f"NA: Order: {na_order}, Family: {na_family}, Genus: {na_genus}, Species: {na_species}")
    print(f"EA: Order: {ea_order}, Family: {ea_family}, Genus: {ea_genus}, Species: {ea_species}")
    print("-" * 50)

NA: Order: Artiodactyla, Family: Antilocapridae, Genus: Antilocapra, Species: americana
EA: Order: Artiodactyla, Family: Bovidae, Genus: Antilope, Species: cervicapra
--------------------------------------------------
NA: Order: Artiodactyla, Family: Bovidae, Genus: Bison, Species: bison
EA: Order: Artiodactyla, Family: Bovidae, Genus: Antilope, Species: cervicapra
--------------------------------------------------
NA: Order: Artiodactyla, Family: Bovidae, Genus: Oreamnos, Species: americanus
EA: Order: Artiodactyla, Family: Bovidae, Genus: Antilope, Species: cervicapra
--------------------------------------------------
NA: Order: Artiodactyla, Family: Bovidae, Genus: Ovibos, Species: moschatus
EA: Order: Artiodactyla, Family: Bovidae, Genus: Antilope, Species: cervicapra
--------------------------------------------------
NA: Order: Artiodactyla, Family: Bovidae, Genus: Ovis, Species: canadensis
EA: Order: Artiodactyla, Family: Bovidae, Genus: Antilope, Species: cervicapra
------------

In [120]:
# Return all animals from EA and NA belonging to the same order.
# This JOINs the data in the two tables together into a single dataset (here in the form of a tuple).
# Note: We are going to limit the results to 10 for printing
sel = [EA.family, EA.genus, EA.species, NA.family, NA.genus, NA.species]
same_order = session.query(*sel).filter(EA.order == NA.order).limit(10).all()

for record in same_order:
    (ea_fam, ea_gen, ea_spec, na_fam, na_gen, na_spec) = record
    print(
        f"The European animal '{ea_fam} {ea_gen} {ea_spec}'"
        f"belongs to the same order as the North American animal '{na_fam} {na_gen} {na_spec}'.")

The European animal 'Bovidae Antilope cervicapra'belongs to the same order as the North American animal 'Antilocapridae Antilocapra americana'.
The European animal 'Bovidae Antilope cervicapra'belongs to the same order as the North American animal 'Antilocapridae Capromeryx mexicana'.
The European animal 'Bovidae Antilope cervicapra'belongs to the same order as the North American animal 'Antilocapridae Capromeryx minor'.
The European animal 'Bovidae Antilope cervicapra'belongs to the same order as the North American animal 'Antilocapridae Stockoceros conklingi'.
The European animal 'Bovidae Antilope cervicapra'belongs to the same order as the North American animal 'Antilocapridae Stockoceros onusrosagris'.
The European animal 'Bovidae Antilope cervicapra'belongs to the same order as the North American animal 'Antilocapridae Tetrameryx shuleri'.
The European animal 'Bovidae Antilope cervicapra'belongs to the same order as the North American animal 'Bovidae Bison bison'.
The European ani